In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn import over_sampling
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import uniform
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

main = pd.read_csv('/content/drive/My Drive/data/application_train.csv')
main
main['SK_ID_CURR'].nunique()
test = pd.read_csv('/content/drive/My Drive/data/application_test.csv')
test
test['SK_ID_CURR'].nunique()

prev = pd.read_csv('/content/drive/My Drive/data/previous_application.csv')
prev

bureau = pd.read_csv('/content/drive/My Drive/data/bureau.csv')
bureau

prev_app = prev.groupby(['SK_ID_CURR'])['SK_ID_CURR'].agg(['count']).reset_index()
prev_app.columns = ['SK_ID_CURR','TOTAL_PREV_APP']
prev_app

bureau_cred = bureau.groupby(['SK_ID_CURR'])['SK_ID_CURR'].agg(['count']).reset_index()
bureau_cred.columns = ['SK_ID_CURR','TOTAL_BUREAU_LOAN']
bureau_cred

df = (main.merge(prev_app, how='right', on='SK_ID_CURR')).merge(bureau_cred, how='left', on='SK_ID_CURR')
df

#data clearning
#duplicat
df.duplicated().sum()

#missing data
total = df.isnull().sum().sort_values(ascending = False)
percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent']).reset_index()
missing_data.loc[missing_data['Percent'] >60]

(missing_data.loc[missing_data['Percent'] >0]).shape
#feature target
df['TARGET'].value_counts()
df['TARGET'].value_counts(normalize=True).plot.bar(figsize=(4,4), title= 'Distribusi Tareget', color=['black','yellow'])
plt.show()

plt.figure(1)
plt.subplot(221)
df['CODE_GENDER'].value_counts(normalize=True).plot.bar(figsize=(8,8), title= 'Client Gender', color=['black','yellow'])
plt.subplot(222)

df['FLAG_OWN_CAR'].value_counts(normalize=True).plot.bar(title= 'Apa client memiliki mobil?', color=['black','yellow'])
plt.subplot(223)

df['CNT_CHILDREN'].value_counts(normalize=True).plot.bar(title= 'Berapa banyak client yang memiliki anak?', color=['black','yellow','blue','red'])
plt.subplot(224)

df['FLAG_OWN_REALTY'].value_counts(normalize=True).plot.bar(figsize=(8,8), title= 'apa client memiliki Realty?', color=['black','yellow'])

plt.show()

df['CODE_GENDER'].value_counts()

df['CNT_CHILDREN'].value_counts()

df['FLAG_OWN_CAR'].value_counts()

df['FLAG_OWN_REALTY'].value_counts()

#Data Preprocessing Data Training
df = (main.merge(prev_app, how='left', on='SK_ID_CURR')).merge(bureau_cred, how='left', on='SK_ID_CURR')
df
#Converting DAYS_BIRTH to years to get client age
df['AGE'] = df['DAYS_BIRTH']/-365
df.drop(columns='DAYS_BIRTH', inplace=True)
data_age = df[['TARGET', 'AGE']]
data_age['AGE_GROUP'] = pd.cut(data_age['AGE'], bins = np.linspace(20, 70, num=6))
data_age = (data_age.groupby(['AGE_GROUP']).mean()).sort_values('TARGET')
data_age
plt.barh(data_age.index.astype(str), round(100*data_age['TARGET']), color='indianred')

plt.ylabel('Age Group (years)')
plt.xlabel('Failure to Repay (%)')
plt.title('Failure to Repay by Age Group');
plt.show()

#Converting DAYS_EMPLOYED to years
df['YEARS_EMPLOYED'] = df['DAYS_EMPLOYED']/-365
df.drop(columns='DAYS_EMPLOYED', inplace=True)

df['YEARS_EMPLOYED'].describe()



df['YEARS_EMPLOYED'].replace({df['YEARS_EMPLOYED'].min(): np.nan}, inplace=True)
df['YEARS_EMPLOYED'].plot.hist(title = 'Years Employment Histogram');
plt.xlabel('Years Employment');

data_employed = df[['TARGET', 'YEARS_EMPLOYED','AGE']]
data_employed['YEARS_EMPLOYED_GROUP'] = pd.cut(data_employed['YEARS_EMPLOYED'], bins = np.linspace(0, 50, num=6))
data_employed = ((data_employed.groupby(['YEARS_EMPLOYED_GROUP']).mean())).sort_values('TARGET')
data_employed



plt.barh(data_employed.index.astype(str), round(100*data_employed['TARGET']), color='indianred')

plt.ylabel('Years Employed Group (years)')
plt.xlabel('Failure to Repay (%)')
plt.title('Failure to Repay by Years Employed Group');
plt.show()

plt.barh(data_employed.index.astype(str), round(data_employed['AGE']), color='indianred')

plt.ylabel('Years Employed Group (years)')
plt.xlabel('Average Age')
plt.title('Average Age vs Years Employed Group');
plt.show()

#Converting DAYS_REGISTRATION to years
df['YEARS_REGISTRATION'] = df['DAYS_REGISTRATION']/-365
df.drop(columns='DAYS_REGISTRATION', inplace=True)

df['YEARS_REGISTRATION'].describe()

data_regist = df[['TARGET', 'YEARS_REGISTRATION']]
data_regist['YEARS_REGISTRATION_GROUP'] = pd.cut(data_regist['YEARS_REGISTRATION'], bins = np.linspace(0, 70, num=8))
data_regist = (data_regist.groupby(['YEARS_REGISTRATION_GROUP']).mean())
data_regist

plt.barh(data_regist.index.astype(str), round(100*data_regist['TARGET']), color='indianred')

plt.ylabel('Years Registration Group (years)')
plt.xlabel('Failure to Repay (%)')
plt.title('Failure to Repay by Years Registration Group');
plt.show()

#Converting DAYS_ID_PUBLISH to years
df['YEARS_ID_PUBLISH'] = df['DAYS_ID_PUBLISH']/-365
df.drop(columns='DAYS_ID_PUBLISH', inplace=True)

df['YEARS_ID_PUBLISH'].describe()
data_id_publish = df[['TARGET', 'YEARS_ID_PUBLISH']]

data_id_publish['YEARS_ID_PUBLISH_GROUP'] = pd.cut(data_id_publish['YEARS_ID_PUBLISH'], bins = np.linspace(0, 20, num=5))
data_id_publish = (data_id_publish.groupby(['YEARS_ID_PUBLISH_GROUP']).mean()).sort_values('TARGET')
data_id_publish

plt.barh(data_id_publish.index.astype(str), round(100*data_id_publish['TARGET']), color='indianred')

plt.ylabel('Change Document Identity Before Application (years)')
plt.xlabel('Failure to Repay (%)')
plt.title('Failure to Repay by DAYS_ID_PUBLISH');
plt.show()

#Replace XNA values with NaN
for col in df.select_dtypes(include = ["object"]).columns:
    print(f'''Value count kolom {col}:''')
    print(df[col].value_counts())
    print()

df.CODE_GENDER.replace("XNA", np.nan, inplace=True)
df.ORGANIZATION_TYPE.replace("XNA", np.nan, inplace=True)
df.shape

total = df.isnull().sum().sort_values(ascending = False)
percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent']).reset_index()
missing_data.loc[missing_data['Percent'] >60]

df.drop(columns=list(missing_data['index'].loc[missing_data['Percent'] >60]),inplace=True)
df.shape

pd.set_option("max_columns", None)
df.head()

#Missing Value Imputation
null = df.isnull().sum().reset_index()
null_table = null.loc[null[0] > 0]
null_table.shape

pd.set_option("max_columns", None)
data_null = df[null_table['index'].tolist()]
data_null

data_null.select_dtypes(exclude = ["object"] ).shape[1]
data_null.select_dtypes(include = ["object"] ).shape[1]

def miss_numerical(df):
    
    numerical_features = df.select_dtypes(exclude = ["object"] ).columns
    for f in numerical_features:
        df[f] = df[f].fillna(df[f].median())
    return df

def miss_categorical(df):
    
    categorical_features = df.select_dtypes(include = ["object"]).columns
    for f in categorical_features:
        df[f] = df[f].fillna(df[f].mode()[0])
    return df

def transform_feature(df):
    df = miss_numerical(df)
    df = miss_categorical(df)
    return df

df = transform_feature(df)
pd.set_option("max_columns", None)
df

null = df.isnull().sum().reset_index()
null_table = null.loc[null[0] > 0]
null_table.shape[0]

def encoder(df):
    scaler = MinMaxScaler()
    numerical = df.select_dtypes(exclude = ["object"]).columns
    numerical = numerical[1:]
    features_transform = pd.DataFrame(data=df)
    features_transform[numerical] = scaler.fit_transform(df[numerical])
    display(features_transform.head(n = 5))
    return df

df = encoder(df)
df.shape

df[df.select_dtypes(include = ["object"]).columns].head()
for col in df.select_dtypes(include = ["object"]).columns:
    print(f'''Value count kolom {col}:''')
    print(df[col].value_counts())
    print()

le = LabelEncoder()
le_count = 0

for col in df:
    if df[col].dtype == 'object':
        if len(list(df[col].unique())) <= 2:
            le.fit(df[col])
            df[col] = le.transform(df[col])            
            le_count += 1

           
print('%d columns were label encoded.' % le_count)
df.shape
le_count = 0

for col in df:
    if df[col].dtype == 'object':
        if len(list(df[col].unique())) > 2:
            onehots = pd.get_dummies(df[col])
            df = df.join(onehots)
            df.drop(columns=col, inplace=True)
            le_count += 1
           
print('%d columns were one hot encoded.' % le_count)
df.shape
df.head()

df_train = df.iloc[:,1:]
DF_TRAIN = df_train
DF_TRAIN

df_test = df.iloc[:,1:]
df_test
DF_TEST = df_test
DF_TEST

target = df_train['TARGET']

df_train, df_test = df_train.align(df_test, join = 'inner', axis = 1)
df_train['TARGET'] = target

def encoder(df):
    scaler = MinMaxScaler()
    numerical = df.select_dtypes(exclude = ["object"]).columns
    numerical1 = numerical[2:4]
    numerical2 = numerical[5:]
    numerical_all = numerical1 | numerical2
    features_transform = pd.DataFrame(data=df)
    features_transform[numerical_all] = scaler.fit_transform(df[numerical_all])
    display(features_transform.head(n = 5))
    return df

df = encoder(df)
scaler_AMT_CREDIT = MinMaxScaler()
scaler_AMT_CREDIT.fit(df['AMT_CREDIT'].values.reshape(len(df), 1))

df['AMT_CREDIT'] = scaler_AMT_CREDIT.transform(df['AMT_CREDIT'].values.reshape(len(df), 1))
df.head()
#Modeling
X = df_train.drop(columns = ['TARGET'])
Y = df_train[['TARGET']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2,random_state = 42)
X_train.shape
X_test.shape
y_train.shape
y_test.shape
y_train['TARGET'].value_counts()
x = X_train[[col for col in X_train.columns if (str(X_train[col].dtype) != 'object')]]
y = y_train['TARGET'].values
print(x.shape)
print(y.shape)
x_over_SMOTE, y_over_SMOTE = over_sampling.SMOTE(0.5).fit_resample(x, y)
print('SMOTE')
print(pd.Series(y_over_SMOTE).value_counts())

data_train = x_over_SMOTE
data_train['TARGET'] = pd.DataFrame(y_over_SMOTE)

print('Original')
print(pd.Series(y).value_counts())
print('')
print('SMOTE')
print(pd.Series(y_over_SMOTE).value_counts())
X_train = data_train.drop(columns = ['TARGET'])
y_train = data_train[['TARGET']]

def eval_classification(model, pred, xtrain, ytrain, xtest, ytest):
    print("Accuracy (Test Set): %.2f" % accuracy_score(ytest, pred))
    print("Precision (Test Set): %.2f" % precision_score(ytest, pred))
    print("Recall (Test Set): %.2f" % recall_score(ytest, pred))
    print("F1-Score (Test Set): %.2f" % f1_score(ytest, pred))
    
    fpr, tpr, thresholds = roc_curve(ytest, pred, pos_label=1) # pos_label: label yang kita anggap positive
    print("AUC: %.2f" % auc(fpr, tpr))

#Logistic Regression
logres = LogisticRegression(random_state=42)
logres.fit(X_train, y_train)

# Predict
y_pred_proba_logres = logres.predict_proba(X_test)
y_pred_test_logres = logres.predict(X_test)

eval_classification(logres, y_pred_test_logres, X_train, y_train, X_test, y_test)

Log_ROC_auc = roc_auc_score(y_test, y_pred_test_logres)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_logres [:, 1])

plt.figure()
plt.plot(fpr, tpr, label = "Logistik Regression Model (area = %0.2f)" % Log_ROC_auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.show()

feat_importances = pd.Series(logres.coef_[0], index=X.columns)
ax = feat_importances.nlargest(32).plot(kind='barh', figsize=(10, 8))
ax.invert_yaxis()

plt.xlabel('Score')
plt.ylabel('Feature')
plt.title('Feature Importance Score')
plt.show()

feat_importances.abs().sort_values(ascending = False)

cf = confusion_matrix(y_test, y_pred_test_logres)
cf
y_pred_train_logres = logres.predict(X_train)
y_pred_test_logres = logres.predict(X_test)
print("Precision (Train Set): " +str(precision_score(y_train, y_pred_train_logres)))
print("Precision (Test Set):" +str(precision_score(y_test, y_pred_test_logres)))

#Hyperparameter Tuning
penalty = ['l2','l1','elasticnet']
C = [0.0001, 0.005] 
hyperparameters = dict(penalty=penalty, C=C)

logres = LogisticRegression(random_state=42) # Init Logres dengan Gridsearch, cross validation = 5
logres_tuning = RandomizedSearchCV(logres, hyperparameters, cv=5, random_state=42, scoring='roc_auc')
logres_tuning.fit(X_train, y_train)

y_pred_proba_logres_tuning = logres_tuning.predict_proba(X_test)
y_pred_test_logres_tuning = logres_tuning.predict(X_test)
eval_classification(logres_tuning, y_pred_test_logres_tuning, X_train, y_train, X_test, y_test)
Log_ROC_auc = roc_auc_score(y_test, y_pred_test_logres_tuning)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_logres_tuning [:, 1])

plt.figure()
plt.plot(fpr, tpr, label = "Logistic Regression Tuned Model (area = %0.2f)" % Log_ROC_auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.show()
print('Best algorithm:', logres_tuning.best_estimator_.get_params()['penalty'])
print('Best C:', logres_tuning.best_estimator_.get_params()['C'])
cf = confusion_matrix(y_test, y_pred_test_logres_tuning)
cf
y_pred_train = logres_tuning.predict(X_train)
y_pred_test = logres_tuning.predict(X_test)
print("Precision (Train Set): " +str(precision_score(y_train, y_pred_train)))
print("Precision (Test Set):" +str(precision_score(y_test, y_pred_test)))

xg = XGBClassifier(random_state=42)
xg.fit(X_train, y_train)

# Predict
y_pred_proba_xg = xg.predict_proba(X_test)
y_pred_test_xg = xg.predict(X_test)
eval_classification(xg, y_pred_test_xg, X_train, y_train, X_test, y_test)
Log_ROC_auc = roc_auc_score(y_test, y_pred_test_xg)
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_xg [:, 1])

plt.figure()
plt.plot(fpr, tpr, label = "XGBoost Model (area = %0.2f)" % Log_ROC_auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.show()
cf = confusion_matrix(y_test, y_pred_test_xg)
cf

DF_TRAIN.shape
DF_TRAIN.head()

DF_TRAIN['AMT_CREDIT'] = scaler_AMT_CREDIT.inverse_transform(DF_TRAIN['AMT_CREDIT'].values.reshape(len(DF_TRAIN), 1))
DF_TRAIN.head()
X = DF_TRAIN.drop(columns = ['TARGET'])
Y = DF_TRAIN[['TARGET']]
XTrain, XTest, yTrain, yTest = train_test_split(X,Y,test_size = 0.2,random_state = 42)
DF_TRAIN.shape
y_pred_test_logres.shape
XTest.shape
XTest['TARGET_PRED_RESULT'] = y_pred_test_logres
XTest['TARGET'] = yTest
XTest.shape
XTest.head()
#sebelum
Defaulters = XTest.groupby(['TARGET'])['AMT_CREDIT'].sum().reset_index()
Defaulters
Total_Defaulters = XTest.groupby(['TARGET'])['AMT_CREDIT'].count().reset_index()
Total_Defaulters
LGDBefore = Defaulters['AMT_CREDIT'].loc[Defaulters['TARGET'] == 1]
LGDBefore
#sesudah
FalseNegative = (XTest.loc[(XTest['TARGET'] == 1) & (XTest['TARGET_PRED_RESULT'] == 0)])
FalseNegative.shape
LGDAfter = FalseNegative['AMT_CREDIT'].sum()
LGDAfter
LGDAfter - LGDBefore
LGD_Decreased_Percentage = ((LGDAfter-LGDBefore)/LGDBefore)*100
LGD_Decreased_Percentage

logres = LogisticRegression(random_state=42)
logres.fit(X_train, y_train)
y_pred_proba_logres_df_test = logres.predict_proba(df_test)
y_pred_proba_logres_df_test
df_test.shape
test.shape
PredictResultTest = test['SK_ID_CURR'].reset_index()
PredictResultTest['TARGET'] = round(pd.DataFrame(y_pred_proba_logres_df_test[:,1]),1)
PredictResultTest.drop('index', axis=1, inplace=True)
PredictResultTest.head()
PredictResultTest.shape
PredictResultTest.to_csv("y_pred_proba_df_test.txt", sep=',', header=True, index=False) 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


count    307511.000000
mean         13.660604
std           9.651743
min          -0.000000
25%           5.506849
50%          12.339726
75%          20.491781
max          67.594521
Name: YEARS_REGISTRATION, dtype: float64

count    307511.000000

mean       -174.835742

std         387.056895

min       -1000.665753

25%           0.791781

50%           3.323288

75%           7.561644

max          49.073973

Name: YEARS_EMPLOYED, dtype: float64

AGE_GROUP,TARGET,AGE

"(60.0, 70.0]",0.049214424239017396,63.35593344850568

"(50.0, 60.0]",0.061297052687255736,54.97657466218351

"(40.0, 50.0]",0.07650801531205498,44.75607944385792

"(30.0, 40.0]",0.09583515575642708,35.123196833011455

"(20.0, 30.0]",0.11456875680238111,26.622040734110993

Y    205732

N     85325

Name: FLAG_OWN_REALTY, dtype: int64

F      192765

M       98288

---


XNA         4

Name: CODE_GENDER, dtype: int64::teks tebal:

0.0     202784

1.0      58450

2.0      25689

3.0       3594

4.0        418

5.0         81

6.0         20

7.0          7

14.0         3

8.0          2

9.0          2

12.0         2

10.0         2

19.0         2

11.0         1

Name: CNT_CHILDREN, dtype: int64

N    192556

Y     98501

Name: FLAG_OWN_CAR, dtype: int64